# Лабораторная работа №3

Кутузов Дмитрий, 21ПИ-1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("TrainData_new.csv")
test = pd.read_csv("TestData_new.csv")

In [ ]:
train.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,cat_feature_1,cat_feature_2,target
0,2.740701,8.506838,47.724081,29.151119,NaN,14.293323,2.255775,51.659614,30.097626,69.505159,1.710745,88.430339,21.172353,16.403757,B,individuals,0
1,1.330506,1.641074,19.835795,25.552527,34.141593,26.684477,23.762340,8.319554,14.627584,35.962469,37.903265,71.587453,5.078505,39.725969,A,individuals,0
2,4.334999,0.415631,39.610327,8.303514,5.141777,86.943615,37.501950,48.213628,15.251909,9.587599,56.865540,40.130363,3.103922,38.253725,C,legal entities,0
3,0.251638,8.136452,16.871775,32.260907,24.638119,47.652316,21.232416,11.024905,48.546775,76.087864,78.080292,631.536797,15.206993,71.217221,C,legal entities,0
4,0.647545,1.753623,2.948700,15.841658,NaN,24.862920,32.185420,21.277126,14.066296,53.271626,11.711993,73.139324,1.896122,76.239624,A,individuals,0


## Часть 1: Работа с пропусками

### Задание 1

Проверьте, есть ли в тренировочных и тестовых данных пропуски? Укажите количество столбцов тренировочной выборки, имеющих пропуски.

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Пропуски есть и в тренеровочных, и в тестовых данных. В тренировочной выборке **14** столбцов имеет пропуски.

### Задание 2

a) В столбце с наибольшим количеством пропусков заполните пропуски средним значением по столбцу. В ответ запишите значение вычисленного среднего. Ответ округлите до десятых.

In [ ]:
print("Cтолбец с наибольшем количеством пропусков:", train.isna().sum().idxmax())

mean_val = train[train.isna().sum().idxmax()].mean()
print("Среднее значение по столбцу c наибольшем количеством пропусков",mean_val)

In [ ]:
column_max_nan_name = train.isna().sum().idxmax()
train[train.isna().sum().idxmax()] = train[train.isna().sum().idxmax()].fillna(mean_val)

In [ ]:
test[column_max_nan_name] = test[column_max_nan_name].fillna(test[test.isna().sum().idxmax()].mean())

b) Найдите строки в тренировочных данных, где пропуски стоят в столбце с наименьшим количеством пропусков. Удалите эти строки. Сколько строк вы удалили?

In [ ]:
column_with_min_null = train.isnull().sum()[train.isnull().sum() > 0].idxmin()
print("Меньше всего пропусков, не считая 0, в столбце:", column_with_min_null)

print("Количество строк для удаления: ", train[train[column_with_min_null].isnull()].shape[0])

In [ ]:
train = train.dropna(subset=[column_with_min_null])

## Часть 2: Предобработка данных

### Задание 3

a) Сколько столбцов в таблице (не считая target) содержат меньше 5 различных значений?

In [ ]:
unique_counts_per_columns = train.drop(columns=['target']).nunique()
print(len(unique_counts_per_columns[unique_counts_per_columns < 5]), "столбцов содержат меньше 5 различ. значений")

2 столбцов содержат меньше 5 различ. значений


b) Вычислите долю ушедших из компании клиентов, для которых значение признака 2 больше среднего значения по столбцу, а значение признака 13 меньше медианы по столбцу. Ответ округлите до сотых.

In [ ]:
mean_feature_2 = train['feature_2'].mean()
median_feature_13 = train['feature_13'].median()

In [ ]:
train[(train['feature_2'] > mean_feature_2) & (train['feature_13'] < median_feature_13)]['target'].mean()

0.09090909090909091

Доля ушедших для таких клиентов: 0.11

## Часть 3: Обучение модели

Перед этим выполним ohe для категориальных признаков, заполним пустые значения в столбцах и нормализуем числовые признаки.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
## для тренировочной

def fit_transform(X, cat_nan_replacer=""):
    scaler = StandardScaler()
    nan_filler = SimpleImputer(strategy="median")
    ohe = OneHotEncoder(drop="first", handle_unknown="ignore")

    cat_features_mask = (X.dtypes == "object").values
    # выделяем числовые признаки
    X_numeric = X[X.columns[~cat_features_mask]]
    # обучаем SimpleImputer
    nan_filler.fit(X_numeric)
    # заполняем пропуски в числовых признаках на тренировочной выборке медианными значениями
    X_num_no_nan = pd.DataFrame(data=nan_filler.transform(X_numeric), columns=X_numeric.columns)
    # обучаем StandardScaler
    scaler.fit(X_num_no_nan)
    # нормализуем числовые данные тренировочной выборки
    X_num_scaled = pd.DataFrame(data=scaler.transform(X_num_no_nan), columns=X_num_no_nan.columns)

    # заполняем пропуски в категориальных признаках
    X_cat = X[X.columns[cat_features_mask]]
    # обучаем OneHotEncoder
    ohe.fit(X_cat)
    # кодируем категориальные признаки
    X_cat_ohe = pd.DataFrame(data=ohe.transform(X_cat).toarray(),  columns=ohe.get_feature_names_out())

    X_prep = pd.concat([X_num_scaled, X_cat_ohe], axis=1)

    return X_prep, scaler, nan_filler, ohe

## для тестовой
def transform(X, scaler, nan_filler, ohe, cat_nan_replacer=""):
    cat_features_mask = (X.dtypes == "object").values

    X_numeric = X[X.columns[~cat_features_mask]]
    # заполняем пропуски в числовых признаках медианными значениями
    X_num_no_nan = pd.DataFrame(data=nan_filler.transform(X_numeric), columns=X_numeric.columns)
    # нормализуем числовые данные
    X_num_scaled = pd.DataFrame(data=scaler.transform(X_num_no_nan), columns=X_num_no_nan.columns)

    # заполняем пропуски в категориальных признаках
    X_cat = X[X.columns[cat_features_mask]]
    # кодируем категориальные признаки
    X_cat_ohe = pd.DataFrame(data=ohe.transform(X_cat).toarray(), columns=ohe.get_feature_names_out())

    X_prep = pd.concat([X_num_scaled, X_cat_ohe], axis=1)

    return X_prep

In [ ]:
X_train = train.drop("target", axis=1)
y_train = train["target"]

X_test = test.drop("target", axis=1)
y_test = test["target"]

In [ ]:
X_train, scaler, nan_filler, ohe = fit_transform(X_train)
X_test = transform(X_test, scaler, nan_filler, ohe)

In [ ]:
X_train.isna().sum()

feature_0                       0
feature_1                       0
feature_2                       0
feature_3                       0
feature_4                       0
feature_5                       0
feature_6                       0
feature_7                       0
feature_8                       0
feature_9                       0
feature_10                      0
feature_11                      0
feature_12                      0
feature_13                      0
cat_feature_1_B                 0
cat_feature_1_C                 0
cat_feature_2_legal entities    0
dtype: int64

### Задание 4

a) Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца target, и матрицу объект-признак X, содержащую остальные признаки. Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1-score алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.
При объявлении модели фиксируйте random_state = 42.
Комментарий: параметры по умолчанию можете оставить дефолтными

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

In [ ]:
model = LogisticRegression(random_state=42)
f1_scorer = make_scorer(f1_score)
f1_scores = cross_val_score(model, X_train, y_train, cv=3, scoring=f1_scorer)

mean_f1_score = np.mean(f1_scores)
print(round(mean_f1_score, 2))

0.82


Среднеезначение метрики f1-score: **0.82**

### Задание 5

a) Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора C примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1-score. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение C.

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию можете оставить дефолтными

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
model = LogisticRegression(random_state=42)

pipeline = Pipeline(steps=[
    ('classifier', model)
])

param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

f1_scorer = make_scorer(f1_score)

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring=f1_scorer)

grid_search.fit(X_train, y_train)

print("Best C:", grid_search.best_params_['classifier__C'])
print('f1-score:', round(grid_search.best_score_, 2))

Best C: 10
f1-score: 0.86


Наилучшее C: **10**

b) Добавьте в тренировочные и тестовые данные новый признак 'NEW', равный произведению признаков '7' и '11'. На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1-score) подберите оптимальное значение C (перебирайте те же значения C, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма (по метрике f1-score), ответ округлите до сотых.

При объявлении модели фиксируйте random_state = 42.


In [ ]:
X_train = train.drop("target", axis=1)
y_train = train["target"]

X_test = test.drop("target", axis=1)
y_test = test["target"]

X_train['NEW'] = X_train['feature_7'] * X_train['feature_11']
X_test['NEW'] = X_test['feature_7'] * X_test['feature_11']

X_train, scaler, nan_filler, ohe = fit_transform(X_train)
X_test = transform(X_test, scaler, nan_filler, ohe)

model = LogisticRegression(random_state=42)

pipeline = Pipeline(steps=[
    ('classifier', model)
])

param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

f1_scorer = make_scorer(f1_score)

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring=f1_scorer)

grid_search.fit(X_train, y_train)

print("Best C:", grid_search.best_params_['classifier__C'])
print("f1-score", round(grid_search.best_score_, 2))

Best C: 10
f1-score 0.86


Наилучшее качество алгоритма (по метрике f1-score): **0.86**

c) Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике F1-Score на тестовых данных.


In [ ]:
train = pd.read_csv("TrainData_new.csv")
test = pd.read_csv("TestData_new.csv")

In [ ]:
X_train = train.drop("target", axis=1)
y_train = train["target"]

X_test = test.drop("target", axis=1)
y_test = test["target"]

In [ ]:
X_train['NEW'] = X_train['feature_7'] * X_train['feature_11']
X_test['NEW'] = X_test['feature_7'] * X_test['feature_11']

In [ ]:
X_train, scaler, nan_filler, ohe = fit_transform(X_train)
X_test = transform(X_test, scaler, nan_filler, ohe)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 1]
}
rf_pipeline = Pipeline(steps=[
    ('classifier', rf_model)
])
rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv=3, scoring=f1_scorer)
rf_grid_search.fit(X_train, y_train)
best_rf_score = rf_grid_search.best_score_
print(best_rf_score)

0.9487857704176489


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.001, 0.01]
}
gb_pipeline = Pipeline(steps=[
    ('classifier', gb_model)
])
gb_grid_search = GridSearchCV(gb_pipeline, gb_param_grid, cv=3, scoring=f1_scorer)
gb_grid_search.fit(X_train, y_train)
best_gb_score = gb_grid_search.best_score_
print(best_gb_score)

0.9491503521354266


////

In [ ]:
best_model = gb_grid_search.best_estimator_

In [ ]:
y_pred_test = best_model.predict(X_test)

In [ ]:
print(f1_score(y_test, y_pred_test))

0.3490566037735849


In [ ]:
y_pred_train = best_model.predict(X_train)

In [ ]:
print(f1_score(y_train, y_pred_train))

1.0
